In [1]:
# check sklearn version
import sklearn
print(sklearn.__version__)

0.21.3


In [2]:
# load data
import pandas as pd
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
windows_filepath = 'D:/Dropbox/Machine Learning/Data/Ouyang/labelData.csv'
mac_filepath = '/Users/jiahuali1991/Dropbox/Machine Learning/Data/Ouyang/labelData.csv'
df = pd.read_csv(mac_filepath,header=None)
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:58:21
--- 0.9854090213775635 seconds ---
End Time = 14:58:22


In [3]:
# concatenate alternating columns with pandas
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

# concatenate alternating columns with pandas
df1 = df.iloc[:,:125]
df1
df2 = df.iloc[:,125:250]
df2

df_new = pd.DataFrame()
for i in range(len(df2.columns)):
    df_new[df1.columns[i]]=df1[df1.columns[i]]
    df_new[df2.columns[i]]=df2[df2.columns[i]]

df_new
df_new.shape


# make X to complex numpy array
import numpy as np
X_reshape = np.array(df_new).reshape(10657,1,125,2)
complex_df = np.apply_along_axis(lambda args: [complex(*args)], 3, X_reshape)
complex_df_reshape = complex_df.reshape(10657,125)

# Fast Fourier Transformation
from scipy.fftpack import fft
ft_df = fft(complex_df_reshape)
ft_real = pd.DataFrame(ft_df.real)
ft_imag = pd.DataFrame(ft_df.imag)
X_full = pd.concat([ft_real, ft_imag],axis=1)

X_full
X_full.to_csv('/Users/jiahuali1991/Dropbox/Machine Learning/Data/Ouyang/labelData_fft.csv', index=False)

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:58:22
--- 16.056145191192627 seconds ---
End Time = 14:58:38


In [4]:
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

# load fft transformed data
import pandas as pd
windows_filepath_fft = 'D:/Dropbox/Machine Learning/Data/Ouyang/labelData_fft.csv'
mac_filepath_fft = '/Users/jiahuali1991/Dropbox/Machine Learning/Data/Ouyang/labelData_fft.csv'
df_fft = pd.read_csv(mac_filepath_fft,header=0)

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:58:38
--- 1.123244047164917 seconds ---
End Time = 14:58:39


In [5]:
# Original Data
X = df.iloc[:,:250]
y = df[250]
#print(X.describe())
# print(X.shape)

# get unique value for the label
import numpy as np
#np.unique(y)
#print(y.describe())

# FFT
# split features (X) and lables (y)
X_fft = df_fft
#print(X_fft.describe())
#print(X_fft.shape)

In [6]:
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

# standardize the data as z-scores
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.astype('float64'))
X_fft_scaled = scaler.fit_transform(X_fft.astype('float64'))

# split training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
X_fft_train, X_fft_test, y_train, y_test = train_test_split(X_fft_scaled, y, test_size=0.33, random_state=42)
y_train = y_train - 1
y_test = y_test -1

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:58:40
--- 1.8615951538085938 seconds ---
End Time = 14:58:41


In [7]:
# deep learning on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

import tensorflow as tf
from tensorflow import keras
tf.__version__
keras.__version__

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[250]))
model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

print(model.summary())

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"]) # sgd

filepath = '/Users/jiahuali1991/Dropbox/Machine Learning/Data/Ouyang/DeepNet_Model_labelData_original.h5'
# model.load_weights(filepath) #load previously trained model


history = model.fit(X_train, y_train, epochs=500, batch_size = 50, verbose = 0,
                    validation_data=(X_test, y_test),
                   callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')
    ])

# we re-load the best weights once training is finished
model.load_weights(filepath)
'''
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()
'''

y_test_pred_dl = model.predict_classes(X_test)
# model.evaluate(X_fft_test, y_test)
from sklearn.metrics import accuracy_score  
print("Deep Learning Model Accuracy: %.2f%%" % (accuracy_score(y_test, y_test_pred_dl) * 100.0))

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:58:41


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               128512    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                5

In [8]:
# deep learning on fft tranformed data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

import tensorflow as tf
from tensorflow import keras
tf.__version__
keras.__version__

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[250]))
model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

print(model.summary())

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"]) # sgd

filepath = '/Users/jiahuali1991/Dropbox/Machine Learning/Data/Ouyang/DeepNet_Model_labelData_fft.h5'
# model.load_weights(filepath) #load previously trained model


history = model.fit(X_fft_train, y_train, epochs=500, batch_size = 50, verbose = 0,
                    validation_data=(X_fft_test, y_test),
                   callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')
    ])

# we re-load the best weights once training is finished
model.load_weights(filepath)
'''
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()
'''
y_test_pred_dl = model.predict_classes(X_fft_test)
# model.evaluate(X_fft_test, y_test)
from sklearn.metrics import accuracy_score  
print("Deep Learning Model Accuracy: %.2f%%" % (accuracy_score(y_test, y_test_pred_dl) * 100.0))

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:59:23
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               128512    
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (No

In [9]:
# Random Forest on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Random Forest Model')    
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_jobs=-1)
rf_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_rf = rf_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
rf_accuracy = accuracy_score(y_test, y_test_pred_rf)
print("Random Forest Model Accuracy: %.2f%%" % (rf_accuracy * 100.0))

# save the model to disk
import pickle
filename = 'RandomForest_model_original.sav'
pickle.dump(rf_clf, open(filename, 'wb'))

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:59:48
start fitting Random Forest Model


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


start making prediction
start calculating accuracy score
Random Forest Model Accuracy: 34.46%
--- 1.1020739078521729 seconds ---
End Time = 14:59:49


In [10]:
# Random Forest on fft transformed data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Random Forest Model')    
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_jobs=-1)
rf_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_rf = rf_clf.predict(X_fft_test)
from sklearn.metrics import accuracy_score  
print("Random Forest Model Accuracy: %.2f%%" % (accuracy_score(y_test, y_test_pred_rf) * 100.0))

# save the model to disk
import pickle
filename = 'RandomForest_model_fft.sav'
pickle.dump(rf_clf, open(filename, 'wb'))

print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")

Start Time = 14:59:49
start fitting Random Forest Model


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


start making prediction
Random Forest Model Accuracy: 40.83%
--- 0.9935691356658936 seconds ---


In [11]:
# Classification and Regression Trees (CART) on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting CART Model')    
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_dt = dt_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
dt_accuracy = accuracy_score(y_test, y_test_pred_dt)
print("CART Model Accuracy: %.2f%%" % (dt_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'CART_model_original.sav'
pickle.dump(dt_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:59:50
start fitting CART Model
start making prediction
start calculating accuracy score
CART Model Accuracy: 28.77%
--- 3.926999092102051 seconds ---
End Time = 14:59:54


In [12]:
# Classification and Regression Trees (CART) on fft data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting CART Model')    
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_dt = dt_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
dt_accuracy = accuracy_score(y_test, y_test_pred_dt)
print("CART Model Accuracy: %.2f%%" % (dt_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'CART_model_fft.sav'
pickle.dump(dt_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 14:59:54
start fitting CART Model
start making prediction
start calculating accuracy score
CART Model Accuracy: 35.71%
--- 3.5552380084991455 seconds ---
End Time = 14:59:58


In [27]:
# KNN on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting KNN Model')    
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_neighbors=17, p=2,
                     weights='distance', n_jobs = -1) 
knn_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_knn = knn_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
knn_accuracy = accuracy_score(y_test, y_test_pred_knn)
print("KNN Model Accuracy: %.2f%%" % (knn_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'KNN_model_original.sav'
pickle.dump(knn_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:28:48
start fitting KNN Model
start making prediction
start calculating accuracy score
KNN Model Accuracy: 36.65%
--- 6.553386926651001 seconds ---
End Time = 15:28:54


In [28]:
# KNN on fft transformed data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting KNN Model')    
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_neighbors=17, p=2,
                     weights='distance', n_jobs = -1) 
knn_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_knn = knn_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
knn_accuracy = accuracy_score(y_test, y_test_pred_knn)
print("KNN Model Accuracy: %.2f%%" % (knn_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'KNN_model_fft.sav'
pickle.dump(knn_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:28:54
start fitting KNN Model
start making prediction
start calculating accuracy score
KNN Model Accuracy: 41.65%
--- 8.47330117225647 seconds ---
End Time = 15:29:03


In [15]:
# Naive Bayes on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Gaussian Naive Bayes Model')    
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_gnb = gnb_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
gnb_accuracy = accuracy_score(y_test, y_test_pred_gnb)
print("Gaussian Naive Bayes Model Accuracy: %.2f%%" % (gnb_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'NB_model_original.sav'
pickle.dump(gnb_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:04
start fitting Gaussian Naive Bayes Model
start making prediction
start calculating accuracy score
Gaussian Naive Bayes Model Accuracy: 9.10%
--- 0.30109500885009766 seconds ---
End Time = 15:00:05


In [16]:
# Naive Bayes
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Gaussian Naive Bayes Model')    
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_gnb = gnb_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
gnb_accuracy = accuracy_score(y_test, y_test_pred_gnb)
print("Random Forest Model Accuracy: %.2f%%" % (gnb_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'NB_model_fft.sav'
pickle.dump(gnb_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:05
start fitting Gaussian Naive Bayes Model
start making prediction
start calculating accuracy score
Random Forest Model Accuracy: 17.34%
--- 0.13392996788024902 seconds ---
End Time = 15:00:05


In [17]:
# Logistic Regression on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Logistic Regression Model')    
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(random_state=0)
lr_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_lr = lr_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
lr_accuracy = accuracy_score(y_test, y_test_pred_lr)
print("Logistic Regression Model Accuracy: %.2f%%" % (lr_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'LR_model_original.sav'
pickle.dump(lr_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:05
start fitting Logistic Regression Model


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


start making prediction
start calculating accuracy score
Logistic Regression Model Accuracy: 16.04%
--- 16.432945013046265 seconds ---
End Time = 15:00:21


In [18]:
# Logistic Regression
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Logistic Regression Model')    
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(random_state=0)
lr_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_lr = lr_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
lr_accuracy = accuracy_score(y_test, y_test_pred_lr)
print("Logistic Regression Model Accuracy: %.2f%%" % (lr_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'LR_model_fft.sav'
pickle.dump(lr_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:21
start fitting Logistic Regression Model


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


start making prediction
start calculating accuracy score
Logistic Regression Model Accuracy: 16.41%
--- 25.17280626296997 seconds ---
End Time = 15:00:47


In [19]:
# SGD on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting SGD Model') 
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_sgd = sgd_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
sgd_accuracy = accuracy_score(y_test, y_test_pred_sgd)
print("SGD Model Accuracy: %.2f%%" % (sgd_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'SGD_model_original.sav'
pickle.dump(sgd_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:47
start fitting SGD Model
start making prediction
start calculating accuracy score
SGD Model Accuracy: 11.91%
--- 2.6764888763427734 seconds ---
End Time = 15:00:49


In [20]:
# SGD on Fourier Tranformed Data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting SGD Model') 
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier()
sgd_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_sgd = sgd_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
sgd_accuracy = accuracy_score(y_test, y_test_pred_sgd)
print("SGD Model Accuracy: %.2f%%" % (sgd_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'SGD_model_fft.sav'
pickle.dump(sgd_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:49
start fitting SGD Model
start making prediction
start calculating accuracy score
SGD Model Accuracy: 14.19%
--- 3.3317391872406006 seconds ---
End Time = 15:00:53


In [21]:
# Light GBM on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Light GBM Model') 
import lightgbm as lgb
gbm = lgb.LGBMClassifier()
gbm.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_gbm = gbm.predict(X_test)
print('start calculating accuracy score') 
gbm_accuracy = accuracy_score(y_test, y_test_pred_gbm)
print("Light GBM Model Accuracy: %.2f%%" % (gbm_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'LightGBM_model_original.sav'
pickle.dump(gbm, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:00:53
start fitting Light GBM Model


/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


start making prediction
start calculating accuracy score
Light GBM Model Accuracy: 38.47%
--- 33.45053195953369 seconds ---
End Time = 15:01:26


In [22]:
# Light GBM on TTF data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Light GBM Model') 
import lightgbm as lgb
gbm = lgb.LGBMClassifier()
gbm.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_gbm = gbm.predict(X_fft_test)
print('start calculating accuracy score') 
gbm_accuracy = accuracy_score(y_test, y_test_pred_gbm)
print("Light GBM Model Accuracy: %.2f%%" % (gbm_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'LightGBM_model_fft.sav'
pickle.dump(gbm, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:01:26
start fitting Light GBM Model
start making prediction
start calculating accuracy score
Light GBM Model Accuracy: 51.27%
--- 36.946532011032104 seconds ---
End Time = 15:02:03


In [23]:
# XGBoost on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting XGBoost Model') 
from xgboost import XGBClassifier
XGB_clf = XGBClassifier()
XGB_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_XGB = XGB_clf.predict(X_test)
print('start calculating accuracy score') 
XGB_accuracy = accuracy_score(y_test, y_test_pred_XGB)
print("XGBoost Model Accuracy: %.2f%%" % (XGB_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'XGBoost_model_original.sav'
pickle.dump(XGB_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:02:03
start fitting XGBoost Model
start making prediction
start calculating accuracy score
XGBoost Model Accuracy: 34.77%
--- 204.981281042099 seconds ---
End Time = 15:05:28


In [24]:
# XGBoost
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting XGBoost Model') 
from xgboost import XGBClassifier
XGB_clf = XGBClassifier()
XGB_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_XGB = XGB_clf.predict(X_fft_test)
print('start calculating accuracy score') 
XGB_accuracy = accuracy_score(y_test, y_test_pred_XGB)
print("XGBoost Model Accuracy: %.2f%%" % (XGB_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'XGBoost_model_fft.sav'
pickle.dump(XGB_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:05:28
start fitting XGBoost Model
start making prediction
start calculating accuracy score
XGBoost Model Accuracy: 44.98%
--- 299.6336889266968 seconds ---
End Time = 15:10:28


In [25]:
# CatBoost on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
import time
start_time = time.time() 
print('start fitting CatBoost Model') 
from catboost import CatBoostClassifier
catb_clf = CatBoostClassifier()
catb_clf.fit(X_train, y_train, verbose=0)
print('start making prediction') 
y_test_pred_catb = catb_clf.predict(X_test)
from sklearn.metrics import accuracy_score
print('start calculating accuracy score') 
catb_accuracy = accuracy_score(y_test, y_test_pred_catb)
print("Cat Boost Model Accuracy: %.2f%%" % (catb_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'CatBoost_model_original.sav'
pickle.dump(catb_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:10:28
start fitting CatBoost Model
start making prediction
start calculating accuracy score
Cat Boost Model Accuracy: 39.52%
--- 519.4524958133698 seconds ---
End Time = 15:19:08


In [26]:
# CatBoost on FFT data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
import time
start_time = time.time() 
print('start fitting CatBoost Model') 
from catboost import CatBoostClassifier
catb_clf = CatBoostClassifier()
catb_clf.fit(X_fft_train, y_train, verbose=0)
print('start making prediction') 
y_test_pred_catb = catb_clf.predict(X_fft_test)
from sklearn.metrics import accuracy_score
print('start calculating accuracy score') 
catb_accuracy = accuracy_score(y_test, y_test_pred_catb)
print("Cat Boost Model Accuracy: %.2f%%" % (catb_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'CatBoost_model_fft.sav'
pickle.dump(catb_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:19:08
start fitting CatBoost Model
start making prediction
start calculating accuracy score
Cat Boost Model Accuracy: 51.07%
--- 579.9042189121246 seconds ---
End Time = 15:28:47


In [32]:
# Support Vector Machine on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Support Vector Machine Model') 
from sklearn import svm
svm_clf = svm.SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
svm_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_svm = svm_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
svm_accuracy = accuracy_score(y_test, y_test_pred_svm)
print("Support Vector Machine Model Accuracy: %.2f%%" % (svm_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'SVM_model_original.sav'
pickle.dump(svm_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 15:58:30
start fitting Support Vector Machine Model
start making prediction
start calculating accuracy score
Support Vector Machine Model Accuracy: 33.10%
--- 125.57747626304626 seconds ---
End Time = 16:00:36


In [33]:
# Support Vector Machine on fft transformed data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting Support Vector Machine Model') 
from sklearn import svm
svm_clf = svm.SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
svm_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_svm = svm_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
svm_accuracy = accuracy_score(y_test, y_test_pred_svm)
print("Support Vector Machine Model Accuracy: %.2f%%" % (svm_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'SVM_model_fft.sav'
pickle.dump(svm_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 16:00:36
start fitting Support Vector Machine Model
start making prediction
start calculating accuracy score
Support Vector Machine Model Accuracy: 35.37%
--- 133.84245705604553 seconds ---
End Time = 16:02:50


In [36]:
# AdaBoost on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting AdaBoost Model') 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=200, algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)
print('start making prediction') 
y_test_pred_ada = ada_clf.predict(X_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
ada_accuracy = accuracy_score(y_test, y_test_pred_ada)
print("AdaBoost Model Accuracy: %.2f%%" % (ada_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'AdaBoost_model_original.sav'
pickle.dump(ada_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 16:04:42
start fitting AdaBoost Model
start making prediction
start calculating accuracy score
AdaBoost Model Accuracy: 28.55%
--- 3.7686259746551514 seconds ---
End Time = 16:04:46


In [37]:
# AdaBoost on original data
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
print('start fitting AdaBoost Model') 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=200, algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_fft_train, y_train)
print('start making prediction') 
y_test_pred_ada = ada_clf.predict(X_fft_test)
print('start calculating accuracy score') 
from sklearn.metrics import accuracy_score  
ada_accuracy = accuracy_score(y_test, y_test_pred_ada)
print("AdaBoost Model Accuracy: %.2f%%" % (ada_accuracy * 100.0))
# save the model to disk
import pickle
filename = 'AdaBoost_model_fft.sav'
pickle.dump(ada_clf, open(filename, 'wb'))
print("--- %s seconds ---" % (time.time() - start_time))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 16:04:46
start fitting AdaBoost Model
start making prediction
start calculating accuracy score
AdaBoost Model Accuracy: 36.05%
--- 3.9777379035949707 seconds ---
End Time = 16:04:50
